In [52]:
import pandas as pd
from io import StringIO
import os
from datetime import datetime, timedelta

from nst_scraper import nst_on_ice_scraper
from db_utils import insert_player, update_player_db, check_last_update, append_player_ids

from dotenv import load_dotenv

pd.set_option('display.max_columns', None)


In [53]:
player_stats_df = nst_on_ice_scraper(fromseason=20242025, thruseason=20242025, startdate='', enddate='')
player_stats_df.head()


,player,team,position,gp,toi,goals,total_assists,first_assists,second_assists,total_points,ipp,shots,sh%,ixg,icf,iff,iscf,ihdcf,rush_attempts,rebounds_created,pim,total_penalties,minor,major,misconduct,penalties_drawn,giveaways,takeaways,hits,hits_taken,shots_blocked,faceoffs_won,faceoffs_lost,faceoffs_%
0,Ryan Suter,STL,D,19,368.366667,1,3,1,2,4,21.05,16,6.25,0.75,45,26,10,2,3,5,12,6,6,0,0,1,21,4,10,14,20,0,0,-
1,Brent Burns,CAR,D,17,277.033333,0,5,2,3,5,33.33,26,0.00,1.12,69,42,19,2,1,5,4,2,2,0,0,1,24,7,2,8,14,0,0,-
2,Corey Perry,EDM,R,19,195.466667,3,0,0,0,3,42.86,17,17.65,2.08,37,25,21,10,0,1,14,4,2,2,0,7,6,3,4,10,5,0,2,0.00
3,Alex Ovechkin,WSH,L,18,215.250000,9,7,4,3,16,72.73,34,26.47,3.27,69,49,40,16,0,5,6,3,3,0,0,0,7,4,41,8,3,0,0,-
4,Evgeni Malkin,PIT,C,20,301.100000,3,9,6,3,12,75.00,29,10.34,4.53,60,39,35,20,0,8,8,4,4,0,0,1,18,3,8,17,14,61,73,45.52


In [54]:
goalie_stats_df = nst_on_ice_scraper(fromseason=20242025, thruseason=20242025, startdate='', enddate='', pos='g')
goalie_stats_df.head()


,player,team,gp,toi,shots_against,saves,goals_against,sv%,gaa,gsaa,xg_against,hd_shots_against,hd_saves,hd_goals_against,hdsv%,hdgaa,hdgsaa,md_shots_against,md_saves,md_goals_against,mdsv%,mdgaa,mdgsaa,ld_shots_against,ld_saves,ld_goals_against,ldsv%,ldgaa,ldgsaa,rush_attempts_against,rebound_attempts_against,avg._shot_distance,avg._goal_distance
0,Marc-Andre Fleury,MIN,4,198.850000,92,85,7,0.924,2.11,1.07,5.51,12,9,3,0.750,0.91,-0.82,23,19,4,0.826,1.21,-1.41,51,51,0,1.000,0.00,1.72,6,10,39.33,20.14
1,Jonathan Quick,NYR,5,225.233333,109,108,1,0.991,0.27,8.56,8.91,27,27,0,1.000,0.00,4.90,32,31,1,0.969,0.27,2.60,47,47,0,1.000,0.00,1.59,11,21,34.87,30.00
2,James Reimer,ANA,2,103.600000,53,48,5,0.906,2.90,-0.35,5.20,7,4,3,0.571,1.74,-1.73,23,21,2,0.913,1.16,0.59,23,23,0,1.000,0.00,0.78,3,8,30.81,17.20
3,Semyon Varlamov,NYI,7,351.650000,152,138,14,0.908,2.39,-0.68,13.49,41,35,6,0.854,1.02,1.44,41,36,5,0.878,0.85,-0.39,55,52,3,0.945,0.51,-1.14,13,24,35.97,20.21
4,Jacob Markstrom,N.J,14,683.216667,317,288,29,0.909,2.55,-1.21,26.59,71,54,17,0.761,1.49,-4.12,88,80,8,0.909,0.70,1.90,138,134,4,0.971,0.35,0.66,41,59,39.39,21.07


In [55]:
class Lineup:
    def __init__(self, name):
        self.name = name
        self.forwards = [None] * 12  # 12 Forward Slots
        self.defense = [None] * 6    # 6 Defense Slots
        self.goalies = [None] * 2    # 2 Goalie Slots

    def add_forward(self, player, slot):
        allowed_positions = {'L', 'C', 'R'}
        if player.position not in allowed_positions:
            raise ValueError(
                f"Cannot add player '{player.name}' with position '{player.position}' to forwards. "
                f"Allowed positions: {', '.join(allowed_positions)}."
            )
        if 0 <= slot < 12:
            if self.forwards[slot] is not None:
                print(f"Warning: Slot {slot + 1} in forwards is already occupied by '{self.forwards[slot].name}'. Overwriting.")
            self.forwards[slot] = player
            print(f"Added player '{player.name}' to forward slot {slot + 1}.")
        else:
            raise IndexError("Forward slot must be between 0 and 11.")

    def add_defense(self, player, slot):
        allowed_position = 'D'
        if player.position != allowed_position:
            raise ValueError(
                f"Cannot add player '{player.name}' with position '{player.position}' to defense. "
                f"Allowed position: '{allowed_position}'."
            )
        if 0 <= slot < 6:
            if self.defense[slot] is not None:
                print(f"Warning: Slot {slot + 1} in defense is already occupied by '{self.defense[slot].name}'. Overwriting.")
            self.defense[slot] = player
            print(f"Added player '{player.name}' to defense slot {slot + 1}.")
        else:
            raise IndexError("Defense slot must be between 0 and 5.")

    def set_goalie(self, player, slot):
        allowed_position = 'G'
        if player.position != allowed_position:
            raise ValueError(
                f"Cannot add player '{player.name}' with position '{player.position}' to goalies. "
                f"Allowed position: '{allowed_position}'."
            )
        if 0 <= slot < 2:
            if self.goalies[slot] is not None:
                print(f"Warning: Goalie slot {slot + 1} is already occupied by '{self.goalies[slot].name}'. Overwriting.")
            self.goalies[slot] = player
            print(f"Added goalie '{player.name}' to goalie slot {slot + 1}.")
        else:
            print(f"Cannot assign goalie to slot {slot}. Maximum of two goalies allowed.")

    def display_lineup(self):
        print(f"Lineup: {self.name}")
        print("Forwards:")
        for idx, player in enumerate(self.forwards):
            print(f"  Slot {idx + 1}: {player if player else 'Empty'}")
        print("Defense:")
        for idx, player in enumerate(self.defense):
            print(f"  Slot {idx + 1}: {player if player else 'Empty'}")
        print("Goalies:")
        for idx, player in enumerate(self.goalies):
            print(f"  Slot {idx + 1}: {player if player else 'Empty'}")
    def to_dataframe(self):
        """
        Converts the lineup into a pandas DataFrame.

        Returns:
            pd.DataFrame: A DataFrame containing forwards, defensemen, and goalies with their respective slots.
        """
        import pandas as pd

        # Prepare data for forwards
        forwards_data = [{
            'Position': 'f',
            'Slot': idx + 1,
            'Player': player.name if player else 'Empty'
        } for idx, player in enumerate(self.forwards)]

        # Prepare data for defensemen
        defense_data = [{
            'Position': 'd',
            'Slot': idx + 1,
            'Player': player.name if player else 'Empty'
        } for idx, player in enumerate(self.defense)]

        # Prepare data for goalies
        goalies_data = [{
            'Position': 'g',
            'Slot': idx + 1,
            'Player': player.name if player else 'Empty'
        } for idx, player in enumerate(self.goalies)]

        # Combine all data
        combined_data = forwards_data + defense_data + goalies_data

        # Create DataFrame
        df = pd.DataFrame(combined_data)

        return df
    def to_transposed_dataframe(self):
        """
        Transposes the lineup DataFrame so that each column represents a position-slot combination
        (e.g., 'f1', 'd1', 'g1') and the row contains the corresponding player names.

        Returns:
            pd.DataFrame: A transposed DataFrame with position-slot as columns and player names as row values.
        """

        # Generate the original DataFrame using the existing method
        df = self.to_dataframe()

        # Initialize an empty dictionary to hold the transposed data
        transposed_data = {}

        # Iterate over each row in the original DataFrame
        for _, row in df.iterrows():
            position = row['Position']
            slot = row['Slot']
            player = row['Player']
            # Create a new column name by combining position and slot number
            column_name = f"{position}{slot}"
            transposed_data[column_name] = player

        # Create a new DataFrame with a single row using the transposed data
        transposed_df = pd.DataFrame([transposed_data])

        return transposed_df

In [56]:
# Define the Player class
class Player:
    def __init__(self, name, team, position, player_id=None):
        """
        Initializes a new Player instance.

        :param name: Full name of the player
        :param team: Team the player belongs to
        :param position: Position of the player (e.g., C, LW, RW, D, G)
        :param player_id: Unique identifier for the player (optional)
        """
        self.player_id = player_id
        self.name = name
        self.team = team
        self.position = position

    def __str__(self):
        """
        Returns a string representation of the player.
        """
        return f"{self.name} ({self.position}) - {self.team}"

    def to_dict(self):
        """
        Converts the Player instance into a dictionary.
        """
        return {
            'player_id': self.player_id,
            'name': self.name,
            'team': self.team,
            'position': self.position
        }


In [57]:
# Creating Player instances from the player_stats_df DataFrame
player_list = []
for _, row in player_stats_df.iterrows():
    player = Player(
        name=row['player'],
        team=row['team'],
        position=row['position']
        # player_id is not set initially
    )
    player_list.append(player)

    # Convert player list to DataFrame
pd.DataFrame([{
    'name': player.name,
    'team': player.team, 
    'position': player.position
} for player in player_list]).head()

,name,team,position
0,Ryan Suter,STL,D
1,Brent Burns,CAR,D
2,Corey Perry,EDM,R
3,Alex Ovechkin,WSH,L
4,Evgeni Malkin,PIT,C


In [58]:
# Creating Player instances from the player_stats_df DataFrame
goalie_list = []
for _, row in goalie_stats_df.iterrows():
    player = Player(
        name=row['player'],
        team=row['team'],
        position='G'
        # player_id is not set initially
    )
    goalie_list.append(player)

    # Convert player list to DataFrame
pd.DataFrame([{
    'name': player.name,
    'team': player.team, 
    'position': player.position
} for player in goalie_list]).head()

,name,team,position
0,Marc-Andre Fleury,MIN,G
1,Jonathan Quick,NYR,G
2,James Reimer,ANA,G
3,Semyon Varlamov,NYI,G
4,Jacob Markstrom,N.J,G


In [59]:
today_datetime= datetime.now()
yesterday_datetime = today_datetime - timedelta(days=1, hours=6) # UTC offset
yesterday = yesterday_datetime.strftime('%Y-%m-%d')
yesterday

'2024-11-18'

In [60]:
# Load environment variables from .env file
load_dotenv()

# Construct the database configuration dictionary
db_config = {
    'dbname': os.getenv('DB_NAME'),
    'user': os.getenv('DB_USER'),
    'password': os.getenv('DB_PASSWORD'),
    'host': os.getenv('DB_HOST'),
    'port': os.getenv('DB_PORT')
}

In [61]:
# Check the last update time of the players database
last_update = check_last_update(db_config)

INFO:db_utils:Database connection established.
INFO:db_utils:Last database update was on: 2024-11-19
INFO:db_utils:Database connection closed.


In [62]:
# Convert last_update to datetime
last_update_dt = datetime.strptime(last_update, '%Y-%m-%d')
today_dt = datetime.strptime(today_datetime.strftime('%Y-%m-%d'), '%Y-%m-%d')
yesterday_dt = datetime.strptime(yesterday, '%Y-%m-%d')

# Only update if last update was before yesterday
if last_update_dt not in [today_dt, yesterday_dt]:
    # Update the player database from last update to yesterday
    update_player_db(last_update, yesterday, db_config, skip_existing=True)
else:
    print(f"No need to update the player database. Last update was on: {last_update}")


No need to update the player database. Last update was on: 2024-11-19


In [63]:
# TODO this function just creates a lineup from the player_list and goalie_list
def create_lineup(team):
    """
    Creates and displays a lineup consisting of players from the specified team.
    
    Args:
        team (str): The team name to filter players.
    """
    # Creating two lineup objects
    lineup1 = Lineup("Lineup 1")
    
    # Adding forwards to lineup1
    forward_count = 0
    for player in player_list:
        if player.team == team:
            try:
                lineup1.add_forward(player, forward_count)
                forward_count += 1
                if forward_count >= 12:
                    break
            except ValueError as e:
                print(f"Skipping player '{player.name}': {e}")
            except IndexError as e:
                print(f"Skipping player '{player.name}': {e}")
        else:
            continue  # Proceed to the next player if not in the specified team
    
    # Adding defense to lineup1
    defense_count = 0
    for player in player_list:
        if player.team == team:
            try:
                lineup1.add_defense(player, defense_count)
                defense_count += 1
                if defense_count >= 6:
                    break
            except ValueError as e:
                print(f"Skipping player '{player.name}': {e}")
            except IndexError as e:
                print(f"Skipping player '{player.name}': {e}")
        else:
            continue  # Proceed to the next player if not in the specified team
    
    # Adding goalies to lineup1
    goalie_count = 0
    for goalie in goalie_list:
        if goalie.team != team:
            continue  # Proceed to the next goalie if not in the specified team
        if goalie_count >= 2:
            print("Maximum of two goalies have been assigned.")
            break
        try:
            lineup1.set_goalie(goalie, goalie_count)
            goalie_count += 1
        except ValueError as e:
            print(f"Skipping goalie '{goalie.name}': {e}")
        except IndexError as e:
            print(f"Skipping goalie '{goalie.name}': {e}")
    
    # Display the lineup
    # lineup1.display_lineup()
    return lineup1

my_lineup = create_lineup('TOR')

Added player 'Ryan Reaves' to forward slot 1.
Added player 'Max Pacioretty' to forward slot 2.
Added player 'John Tavares' to forward slot 3.
Skipping player 'Oliver Ekman-Larsson': Cannot add player 'Oliver Ekman-Larsson' with position 'D' to forwards. Allowed positions: L, R, C.
Skipping player 'Chris Tanev': Cannot add player 'Chris Tanev' with position 'D' to forwards. Allowed positions: L, R, C.
Skipping player 'Jani Hakanpää': Cannot add player 'Jani Hakanpää' with position 'D' to forwards. Allowed positions: L, R, C.
Skipping player 'Morgan Rielly': Cannot add player 'Morgan Rielly' with position 'D' to forwards. Allowed positions: L, R, C.
Skipping player 'Jake McCabe': Cannot add player 'Jake McCabe' with position 'D' to forwards. Allowed positions: L, R, C.
Added player 'Max Domi' to forward slot 4.
Added player 'William Nylander' to forward slot 5.
Added player 'Mitch Marner' to forward slot 6.
Added player 'Steven Lorentz' to forward slot 7.
Skipping player 'Philippe Myers'

In [66]:
my_lineup.to_dataframe()

,Position,Slot,Player
0,f,1,Ryan Reaves
1,f,2,Max Pacioretty
2,f,3,John Tavares
3,f,4,Max Domi
4,f,5,William Nylander
5,f,6,Mitch Marner
6,f,7,Steven Lorentz
7,f,8,Auston Matthews
8,f,9,David Kampf
9,f,10,Connor Dewar


In [64]:
# Convert the lineup to a transposed DataFrame
transposed_lineup_df = my_lineup.to_transposed_dataframe()

# Display the transposed DataFrame
transposed_lineup_df

,f1,f2,f3,f4,f5,f6,f7,f8,f9,f10,f11,f12,d1,d2,d3,d4,d5,d6,g1,g2
0,Ryan Reaves,Max Pacioretty,John Tavares,Max Domi,William Nylander,Mitch Marner,Steven Lorentz,Auston Matthews,David Kampf,Connor Dewar,Pontus Holmberg,Nicholas Robertson,Oliver Ekman-Larsson,Chris Tanev,Jani Hakanpää,Morgan Rielly,Jake McCabe,Philippe Myers,Anthony Stolarz,Joseph Woll


In [37]:
def assign_player_ids_to_lineup(transposed_lineup_df, db_config):
    """
    Processes the transposed_lineup_df DataFrame to append player IDs next to each player's name.
    
    This function performs the following steps:
        1. Extracts all unique player names from the lineup DataFrame.
        2. Creates Player instances for each player name.
        3. Uses the append_player_ids function to assign player IDs to each Player object.
        4. Inserts new columns into the DataFrame with the corresponding player IDs.
    
    Args:
        transposed_lineup_df (pd.DataFrame): The transposed lineup DataFrame with player names.
        db_config (dict): Database configuration with keys: dbname, user, password, host, port.
    
    Returns:
        pd.DataFrame: The updated DataFrame with player IDs added.
    """
    import pandas as pd

    # Ensure the DataFrame has only one row
    if transposed_lineup_df.shape[0] != 1:
        raise ValueError("transposed_lineup_df should contain exactly one row representing the lineup.")

    # Extract player names from the DataFrame
    player_columns = transposed_lineup_df.columns
    player_names = transposed_lineup_df.iloc[0].tolist()

    # Create Player instances
    player_list = []
    for name in player_names:
        if name != 'Empty':
            player = Player(name=name, team=None, position=None)  # Team and position can be set if available
            player_list.append(player)

    # Append player IDs using the existing function
    append_player_ids(player_list, db_config)

    # Create a mapping from player name to player_id
    name_to_id = {player.name: player.player_id for player in player_list if player.player_id is not None}

    # Initialize a copy of the DataFrame to avoid modifying the original
    updated_df = transposed_lineup_df.copy()

    # Insert player_id columns adjacent to each player name column
    for col in player_columns:
        player_name = updated_df.at[0, col]
        if player_name != 'Empty':
            player_id = name_to_id.get(player_name, None)
            id_column = f"{col}_id"
            updated_df[id_column] = player_id
        else:
            id_column = f"{col}_id"
            updated_df[id_column] = None  # Assign None for empty slots

    return updated_df

In [16]:
# Assuming you have already created `transposed_lineup_df` and `db_config`
updated_lineup_df = assign_player_ids_to_lineup(transposed_lineup_df, db_config)

# Display the updated DataFrame
updated_lineup_df

INFO:db_utils:Database connection established.
INFO:db_utils:Assigned player_id 8471817 to Ryan Reaves.
INFO:db_utils:Assigned player_id 8474157 to Max Pacioretty.
INFO:db_utils:Assigned player_id 8475166 to John Tavares.
INFO:db_utils:Assigned player_id 8477503 to Max Domi.
INFO:db_utils:Assigned player_id 8477939 to William Nylander.
INFO:db_utils:Assigned player_id 8478483 to Mitch Marner.
INFO:db_utils:Assigned player_id 8478904 to Steven Lorentz.
INFO:db_utils:Assigned player_id 8479318 to Auston Matthews.
INFO:db_utils:Assigned player_id 8480144 to David Kampf.
INFO:db_utils:Assigned player_id 8480980 to Connor Dewar.
INFO:db_utils:Assigned player_id 8480995 to Pontus Holmberg.
INFO:db_utils:Assigned player_id 8481582 to Nicholas Robertson.
INFO:db_utils:Assigned player_id 8475171 to Oliver Ekman-Larsson.
INFO:db_utils:Assigned player_id 8475690 to Chris Tanev.
INFO:db_utils:Assigned player_id 8475825 to Jani Hakanpää.
INFO:db_utils:Assigned player_id 8476853 to Morgan Rielly.
IN

,f1,f2,f3,f4,f5,f6,f7,f8,f9,f10,f11,f12,d1,d2,d3,d4,d5,d6,g1,g2,f1_id,f2_id,f3_id,f4_id,f5_id,f6_id,f7_id,f8_id,f9_id,f10_id,f11_id,f12_id,d1_id,d2_id,d3_id,d4_id,d5_id,d6_id,g1_id,g2_id
0,Ryan Reaves,Max Pacioretty,John Tavares,Max Domi,William Nylander,Mitch Marner,Steven Lorentz,Auston Matthews,David Kampf,Connor Dewar,Pontus Holmberg,Nicholas Robertson,Oliver Ekman-Larsson,Chris Tanev,Jani Hakanpää,Morgan Rielly,Jake McCabe,Philippe Myers,Anthony Stolarz,Joseph Woll,8471817,8474157,8475166,8477503,8477939,8478483,8478904,8479318,8480144,8480980,8480995,8481582,8475171,8475690,8475825,8476853,8476931,8479026,8476932,8479361


In [19]:
def assign_player_gps_to_lineup_with_goalies(transposed_lineup_df, player_stats_df, goalie_stats_df):
    """
    Processes the transposed_lineup_df DataFrame to append games played (gp) next to each player's name.
    For skaters (Forwards and Defense), 'gp' is retrieved from player_stats_df.
    For goalies, 'gp' is retrieved from goalie_stats_df.
    
    This function performs the following steps:
        1. Iterates through each column in the lineup.
        2. Determines the player type based on the column prefix.
        3. Retrieves the corresponding 'gp' from the appropriate DataFrame.
        4. Inserts a new column adjacent to each player's name containing the 'gp' value.
    
    Args:
        transposed_lineup_df (pd.DataFrame): The transposed lineup DataFrame with player names.
        player_stats_df (pd.DataFrame): DataFrame containing skater statistics, including 'player' and 'gp'.
        goalie_stats_df (pd.DataFrame): DataFrame containing goalie statistics, including 'player' and 'gp'.
    
    Returns:
        pd.DataFrame: The updated DataFrame with 'gp' values added next to each player name.
    """
    import pandas as pd
    
    # Ensure the DataFrame has only one row
    if transposed_lineup_df.shape[0] != 1:
        raise ValueError("transposed_lineup_df should contain exactly one row representing the lineup.")
    
    # Extract player names from the DataFrame
    player_columns = transposed_lineup_df.columns
    player_names = transposed_lineup_df.iloc[0].tolist()
    
    # Create mappings for 'gp' from player_stats_df and goalie_stats_df
    skater_gp_mapping = player_stats_df.set_index('player')['gp'].to_dict()
    goalie_gp_mapping = goalie_stats_df.set_index('player')['gp'].to_dict()
    
    # Initialize a copy of the DataFrame to avoid modifying the original
    updated_df = transposed_lineup_df.copy()
    
    # Iterate through each player column and add a new column with 'gp' value
    for col in player_columns:
        player_name = updated_df.at[0, col]
        gp_column = f"{col}_gp"
        
        if player_name != 'Empty':
            # Determine the player type based on column prefix
            if col.startswith(('f', 'd')):
                # Skater (Forward or Defense)
                gp_value = skater_gp_mapping.get(player_name, None)
                if gp_value is None:
                    print(f"Warning: 'gp' not found for skater '{player_name}'. Assigning None.")
            elif col.startswith('g'):
                # Goalie
                gp_value = goalie_gp_mapping.get(player_name, None)
                if gp_value is None:
                    print(f"Warning: 'gp' not found for goalie '{player_name}'. Assigning None.")
            else:
                # Unknown position prefix
                print(f"Warning: Column '{col}' does not start with F, D, or G. Unable to determine player type.")
                gp_value = None
            
            updated_df[gp_column] = gp_value
        else:
            updated_df[gp_column] = None  # Assign None for empty slots
    
    return updated_df

In [20]:
# Assuming you have already created `transposed_lineup_df`, `player_stats_df`, and `goalie_stats_df`

# Assign games played (gp) to both skaters and goalies
updated_lineup_with_gp_df = assign_player_gps_to_lineup_with_goalies(
    transposed_lineup_df,
    player_stats_df,
    goalie_stats_df
)

# Display the updated DataFrame
updated_lineup_with_gp_df

,f1,f2,f3,f4,f5,f6,f7,f8,f9,f10,f11,f12,d1,d2,d3,d4,d5,d6,g1,g2,f1_gp,f2_gp,f3_gp,f4_gp,f5_gp,f6_gp,f7_gp,f8_gp,f9_gp,f10_gp,f11_gp,f12_gp,d1_gp,d2_gp,d3_gp,d4_gp,d5_gp,d6_gp,g1_gp,g2_gp
0,Ryan Reaves,Max Pacioretty,John Tavares,Max Domi,William Nylander,Mitch Marner,Steven Lorentz,Auston Matthews,David Kampf,Connor Dewar,Pontus Holmberg,Nicholas Robertson,Oliver Ekman-Larsson,Chris Tanev,Jani Hakanpää,Morgan Rielly,Jake McCabe,Philippe Myers,Anthony Stolarz,Joseph Woll,16,13,18,19,19,19,19,13,18,3,17,16,19,19,2,19,19,1,12,5


In [18]:
# player_stats_df = nst_on_ice_scraper(fromseason=20242025, thruseason=20242025, startdate='', enddate='')
player_stats_df.head() 

,player,team,position,gp,toi,goals,total_assists,first_assists,second_assists,total_points,ipp,shots,sh%,ixg,icf,iff,iscf,ihdcf,rush_attempts,rebounds_created,pim,total_penalties,minor,major,misconduct,penalties_drawn,giveaways,takeaways,hits,hits_taken,shots_blocked,faceoffs_won,faceoffs_lost,faceoffs_%
0,Ryan Suter,STL,D,19,368.366667,1,3,1,2,4,21.05,16,6.25,0.75,45,26,10,2,3,5,12,6,6,0,0,1,21,4,10,14,20,0,0,-
1,Brent Burns,CAR,D,17,277.033333,0,5,2,3,5,33.33,26,0.00,1.12,69,42,19,2,1,5,4,2,2,0,0,1,24,7,2,8,14,0,0,-
2,Corey Perry,EDM,R,18,182.616667,3,0,0,0,3,42.86,16,18.75,2.00,34,24,20,9,0,1,14,4,2,2,0,6,6,3,4,10,4,0,2,0.00
3,Alex Ovechkin,WSH,L,17,206.166667,8,7,4,3,15,75.00,32,25.00,3.11,65,46,39,16,0,5,4,2,2,0,0,0,7,4,40,8,3,0,0,-
4,Evgeni Malkin,PIT,C,20,301.100000,3,9,6,3,12,75.00,29,10.34,4.53,60,39,35,20,0,8,8,4,4,0,0,1,18,3,8,17,14,61,73,45.52


In [10]:
from pbp import get_team_roster

get_team_roster('TOR', 20232024)

{'forwards': [{'id': 8481720,
   'headshot': 'https://assets.nhle.com/mugs/nhl/20232024/TOR/8481720.png',
   'firstName': {'default': 'Nick',
    'cs': 'Nicholas',
    'de': 'Nicholas',
    'es': 'Nicholas',
    'fi': 'Nicholas',
    'sk': 'Nicholas',
    'sv': 'Nicholas'},
   'lastName': {'default': 'Abruzzese'},
   'sweaterNumber': 26,
   'positionCode': 'C',
   'shootsCatches': 'L',
   'heightInInches': 71,
   'weightInPounds': 180,
   'heightInCentimeters': 180,
   'weightInKilograms': 82,
   'birthDate': '1999-06-04',
   'birthCity': {'default': 'Slate Hill'},
   'birthCountry': 'USA',
   'birthStateProvince': {'default': 'NY'}},
  {'id': 8480980,
   'headshot': 'https://assets.nhle.com/mugs/nhl/20232024/TOR/8480980.png',
   'firstName': {'default': 'Connor'},
   'lastName': {'default': 'Dewar'},
   'sweaterNumber': 24,
   'positionCode': 'C',
   'shootsCatches': 'L',
   'heightInInches': 70,
   'weightInPounds': 183,
   'heightInCentimeters': 178,
   'weightInKilograms': 83,
   '

In [13]:
from pbp_utils import get_matchup_games

games = get_matchup_games('2024-11-11', '2024-11-17')
pd.DataFrame({
    'game_id': games['game_ids']['id'],
    'date': games['game_ids']['date']
}).head()


,game_id,date
0,2024020239,2024-11-11
1,2024020240,2024-11-11
2,2024020241,2024-11-11
3,2024020242,2024-11-11
4,2024020243,2024-11-11


In [8]:
from game_utils import get_game_boxscore, display_boxscore

temp_data = get_game_boxscore(2024020255)

In [9]:
away_skaters, away_goalies, home_skaters, home_goalies = display_boxscore(temp_data)
away_skaters


,playerId,sweaterNumber,name,position,goals,assists,points,plusMinus,pim,hits,powerPlayGoals,sog,faceoffWinningPctg,toi,blockedShots,shifts,giveaways,takeaways,team
0,8477960,9,A. Kempe,R,2,0,2,0,0,1,0,4,0.000000,19:11,1,23,0,0,Away
1,8471685,11,A. Kopitar,C,0,2,2,0,0,0,0,1,0.769231,19:29,0,23,0,0,Away
2,8479675,12,T. Moore,L,0,2,2,0,0,1,0,0,0.000000,18:52,1,24,0,0,Away
3,8482155,14,A. Laferriere,R,0,0,0,-3,0,2,0,1,0.000000,15:46,0,22,0,1,Away
4,8477942,22,K. Fiala,L,0,0,0,-1,2,2,0,1,0.000000,18:00,2,22,2,0,Away
5,8476479,24,P. Danault,C,0,0,0,-1,2,1,0,0,0.636364,18:11,1,25,1,1,Away
6,8480851,26,A. Thomas,C,0,0,0,0,0,1,0,0,0.000000,11:09,1,15,0,0,Away
7,8477998,37,W. Foegele,L,0,0,0,-1,0,2,0,1,0.000000,15:37,1,25,2,0,Away
8,8481732,47,A. Lee,L,0,0,0,-1,0,3,0,2,0.000000,10:43,2,13,0,1,Away
9,8482124,55,Q. Byfield,R,0,0,0,-1,0,1,0,1,0.500000,13:46,0,22,2,0,Away


In [10]:
away_goalies

,playerId,sweaterNumber,name,position,evenStrengthShotsAgainst,powerPlayShotsAgainst,shorthandedShotsAgainst,saveShotsAgainst,savePctg,evenStrengthGoalsAgainst,powerPlayGoalsAgainst,shorthandedGoalsAgainst,pim,goalsAgainst,toi,starter,decision,shotsAgainst,saves,team
0,8479496,31,D. Rittich,G,4/5,0/0,0/0,4/5,0.8,1,0,0,0,1,14:41,False,L,5,4,Away
1,8475311,35,D. Kuemper,G,17/19,1/1,0/0,18/20,0.9,2,0,0,0,2,44:32,True,NaN,20,18,Away


In [11]:
home_skaters

,playerId,sweaterNumber,name,position,goals,assists,points,plusMinus,pim,hits,powerPlayGoals,sog,faceoffWinningPctg,toi,blockedShots,shifts,giveaways,takeaways,team
0,8476391,9,T. Tynan,C,0,0,0,0,0,0,0,0,0.000000,07:39,1,12,1,0,Home
1,8475780,14,C. Wagner,R,0,0,0,0,0,0,0,0,0.000000,04:24,0,10,0,0,Home
2,8480448,17,P. Kelly,C,0,0,0,0,0,3,0,1,0.555556,13:12,2,21,1,0,Home
3,8481186,25,L. O'Connor,R,0,1,1,1,0,1,0,0,0.000000,15:56,0,23,1,0,Home
4,8477492,29,N. MacKinnon,C,0,3,3,3,0,1,0,3,0.272727,24:45,0,27,1,0,Home
5,8479999,37,C. Mittelstadt,C,0,0,0,-1,0,0,0,1,0.428571,18:35,0,22,3,2,Home
6,8481042,51,N. Kovalenko,R,0,0,0,-1,2,2,0,0,0.000000,14:57,0,19,0,0,Home
7,8477476,62,A. Lehkonen,L,1,1,2,2,0,1,0,3,0.000000,23:13,1,25,0,0,Home
8,8483930,82,I. Ivan,C,0,0,0,-1,0,1,0,1,0.000000,16:18,1,19,0,0,Home
9,8485105,85,N. Prishchepov,C,0,0,0,0,0,1,0,1,0.000000,04:23,0,8,0,0,Home


In [14]:
home_goalies

,playerId,sweaterNumber,name,position,evenStrengthShotsAgainst,powerPlayShotsAgainst,shorthandedShotsAgainst,saveShotsAgainst,savePctg,evenStrengthGoalsAgainst,powerPlayGoalsAgainst,shorthandedGoalsAgainst,pim,goalsAgainst,toi,starter,decision,shotsAgainst,saves,team
0,8480382,40,A. Georgiev,G,11/13,2/2,0/0,13/15,0.866667,2,0,0,0,2,60:00,True,W,15,13,Home
1,8481020,60,J. Annunen,G,0/0,0/0,0/0,0/0,NaN,0,0,0,0,0,00:00,False,NaN,0,0,Home


In [17]:
lineup2.display_lineup()


Lineup: Lineup 2
Forwards:
  Slot 1: Empty
  Slot 2: Empty
  Slot 3: Empty
  Slot 4: Empty
  Slot 5: Empty
  Slot 6: Empty
  Slot 7: Empty
  Slot 8: Empty
  Slot 9: Empty
  Slot 10: Empty
  Slot 11: Empty
  Slot 12: Empty
Defense:
  Slot 1: Empty
  Slot 2: Empty
  Slot 3: Empty
  Slot 4: Empty
  Slot 5: Empty
  Slot 6: Empty
Goalie: Empty

